Welcome to my Capstone project for the IBM Data Science Professional Certification course on [Coursera](https://www.coursera.org/professional-certificates/ibm-data-science). My name is Tyler Kuromiya Parker, and this particular project will focus on using location data. To see more of my work, please check out my github profile located [here](https://www.github.com/tylerkkp), or my [website](https://www.tkkp.dev)

# IBM Data Science Capstone
## Week 4 - Problem Statement

Beer tourism is becoming increasingly common, and (fortunately) there is an abundance of choices for destinations. According to [this article](https://fortune.com/2019/08/06/new-craft-breweries-2019-data/), over 1,000 opened in 2018. That's a bit less than 3 per day! One of the more popular cities for beer tourism is Portland, Oregon. I went to college in Portland, but was underage and did not have the palate for good beer at the time. However, in the years since leaving Portland, I've become increasingly involved in the craft beer industry. Most recently, I started contributing to the OpenBreweryDB project, which seeks to catalog craft breweries across the USA and the world (currently including England and Scotland, although additional countries are anticipated in the future). 

As with many large cities, Portland has quite a few distinct neighborhoods, each with its own unique personality and characteristics. It is also an interesting city in that it is bisected by a river. The industrial and business districts are mostly on the West side of the river, and the neighborhoods are to the East. Breweries are all over the place. However, our aim with this project will be to find the _best_ neighborhood for beer - a rather high honor in a city with so many great breweries.

## The Data

In this project I will be looking at the neighborhoods as defined on the City of Portland Open Data portal [here](https://gis-pdx.opendata.arcgis.com/datasets/neighborhoods-regions?geometry=-123.656%2C45.374%2C-121.678%2C45.711). I will also be using the [OpenBreweryDB](https://www.openbrewerydb.org/) dataset, either through directly downloading the dataset, or using the API. I want to do a couple of analyses. In the spirit of 'Battle of Neighborhoods', I want to see which neighborhood can claim:

1. The most breweries
2. The 'best' breweries based on Foursquare reviews
3. Whatever else I can see from the initial exploratory analysis

The OpenBreweryDB dataset has a number of interesting datapoints which could provide some insights. For example, the predominant 'type' of brewery in a neighborhood. Brewpubs and other similar establishments would likely be more prevalent in tourist areas, and larger breweries would be found in more industrial areas where land and rent are cheaper. I would expect Nano and Micro Breweries to be in neighborhoods since they could be sustained by a small local population. My analysis will help determine if my assumptions hold true. 

In [179]:
!pip install geopandas
!pip install geojson

import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import matplotlib.pyplot as plt
import geojson


# print('Hello Capstone Project Course!')

First, we need to get some data. When we import the data, we will be putting it into DataFrames for further analysis.

In [180]:
# get the geoJSON data from the City of Portland open data portal
url = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'

geometry = gpd.read_file(url)
print(geometry.head())
print(geometry.columns)

   OBJECTID  ...                                           geometry
0         1  ...  POLYGON ((-122.74774 45.57642, -122.74500 45.5...
1         2  ...  POLYGON ((-122.70796 45.57496, -122.70797 45.5...
2         3  ...  POLYGON ((-122.67545 45.58659, -122.67593 45.5...
3         4  ...  POLYGON ((-122.64352 45.56633, -122.64349 45.5...
4         5  ...  POLYGON ((-122.61471 45.54829, -122.60785 45.5...

[5 rows x 11 columns]
Index(['OBJECTID', 'NAME', 'COMMPLAN', 'SHARED', 'COALIT', 'HORZ_VERT',
       'MAPLABEL', 'ID', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')


Ok, how many distinct neighborhoods are we looking at?

In [181]:
print(geometry.shape)
print(geometry['NAME'])

(98, 11)
0                          CATHEDRAL PARK
1                         UNIVERSITY PARK
2                                PIEDMONT
3                                WOODLAWN
4          CULLY ASSOCIATION OF NEIGHBORS
                     ...                 
93    SUNDERLAND ASSOCIATION OF NEIGHBORS
94                             PORTSMOUTH
95     HAYDEN ISLAND NEIGHBORHOOD NETWORK
96                              BRIDGETON
97                          EAST COLUMBIA
Name: NAME, Length: 98, dtype: object


So we have 98 neighborhoods to consider - wow! I suspect many of the outlying neighborhoods may have a very low density of breweries, so we may end up looking more closely at the city's core, but we'll cross that bridge when we get there.

We have our geo shapefile data, but now we need to get our brewery data! The brewery data is from OpenBreweryDB, found [here](www.openbrewerydb.org)

In [182]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [183]:
# for some reason, the below API call was only getting 20 breweries
# burl = 'https://api.openbrewerydb.org/breweries?by_state=oregon'

csv_url = '/content/drive/MyDrive/Coursera IBM Data Science Backup/pdx_brew/oregon.csv'
breweries = pd.read_csv(csv_url)
print(breweries.head())
print(breweries.shape)

                                   id  ... tags
0           10-barrel-brewing-co-bend  ...  NaN
1         10-barrel-brewing-co-bend-2  ...  NaN
2  10-barrel-brewing-co-bend-pub-bend  ...  NaN
3       10-barrel-brewing-co-portland  ...  NaN
4            1188-brewing-co-john-day  ...  NaN

[5 rows x 18 columns]
(291, 18)


Great! Now that we have the brewery database imported, we need to filter out the cities that are not 'Portland'. However, we first need to figure out the names of the columns.

In [184]:
print(breweries.columns)
# check to see if we have null values
needloc = breweries[breweries['longitude'].isna()]
needloc = needloc[needloc['brewery_type'] != 'planning']


Index(['id', 'name', 'brewery_type', 'street', 'address_2', 'address_3',
       'city', 'state', 'county_province', 'postal_code', 'website_url',
       'phone', 'created_at', 'updated_at', 'country', 'longitude', 'latitude',
       'tags'],
      dtype='object')


Now that we have the list of column names, we can create a dataframe containing only the breweries within the city of Portland.

In [185]:
breweries = breweries[breweries['city'] == 'Portland']
# only use breweries with latitude (if there is latitude, there is longitude)
breweries = breweries[breweries['latitude'].isna() == False]
breweries.reset_index(drop=True)
print(breweries.shape)

(67, 18)


We only have 85 breweries for 98 neighborhoods. Slightly disappointing - we will need to map some in order to see where to focus our attention. 

Let's begin with a quick and dirty visualization of the breweries on a folium map.


In [186]:
# Portland, OR has the coordinates of (45.523064, -122.676483)
location = [45.523064, -122.676483]
gdf = geometry.to_crs("EPSG:4326") 
m = folium.Map(location=location, zoom_start=12) # NOTE - may want to use (tiles='cartodbpositron') argument for clarity
folium.GeoJson(data=gdf["geometry"]).add_to(m) 

# add brewery markers
for i in range(0, len(breweries.index)):
  folium.Marker(location=[breweries.iloc[i]['latitude'], breweries.iloc[i]['longitude']], popup=breweries.iloc[i]['name']).add_to(m)

m



Looking good! There are definitely some clusters on the map that are going to be more compelling for beer fans. The downside to this view is that we can't see just _how_ dense the clusters are in each neighborhood. 

To take a closer look at this, we will count the number of breweries in each neighborhood and create a choropleth map. The most brewery-dense neighborhood should be obvious from this visualization. To normalize for differing areas of neighborhoods, we will also need to account for the area of each neighborhood, resulting in a 'breweries per square km' value for each of the 97 neighborhoods. 

We will need to first convert the CRS to EPSG 3857 (which is in meters), then find the area and convert to square kilometers. We can then add this data to a new 'area' column in the gdf dataframe and finally convert back to EPSG 4326

In [187]:
print(gdf.head())

# Change crs to epsg 3857
gdf = gdf.to_crs(epsg = 3857)
print(gdf.crs)
print(gdf.head())

# Create area in square km
sqm_to_sqkm = 10**6
gdf['area'] = gdf.geometry.area / sqm_to_sqkm

# Change gdf crs back to epsg 4326
gdf = gdf.to_crs(epsg = 4326)
print(gdf.crs)
print(gdf.head())

   OBJECTID  ...                                           geometry
0         1  ...  POLYGON ((-122.74774 45.57642, -122.74500 45.5...
1         2  ...  POLYGON ((-122.70796 45.57496, -122.70797 45.5...
2         3  ...  POLYGON ((-122.67545 45.58659, -122.67593 45.5...
3         4  ...  POLYGON ((-122.64352 45.56633, -122.64349 45.5...
4         5  ...  POLYGON ((-122.61471 45.54829, -122.60785 45.5...

[5 rows x 11 columns]
epsg:3857
   OBJECTID  ...                                           geometry
0         1  ...  POLYGON ((-13664216.003 5712727.755, -13663911...
1         2  ...  POLYGON ((-13659787.491 5712495.289, -13659789...
2         3  ...  POLYGON ((-13656168.594 5714345.257, -13656221...
3         4  ...  POLYGON ((-13652613.736 5711123.002, -13652610...
4         5  ...  POLYGON ((-13649407.329 5708255.577, -13648643...

[5 rows x 11 columns]
epsg:4326
   OBJECTID  ...       area
0         1  ...   5.424298
1         2  ...   6.981457
2         3  ...   6.079530
3     

Next we will use a spatial join to find the number of breweries per neighborhood. We start by creating a 'location' value for each brewery, and then 

In [188]:
from shapely.geometry import Point
# below two dependencies are required for the spatial join (gpd.sjoin) to work
!apt install libspatialindex-dev
!pip install rtree

# Create a shapely Point from latitude and longitude
breweries['geometry'] = breweries.apply(lambda x: Point((x.longitude , x.latitude)), axis = 1)

# Build a GeoDataFrame: brew_geo
brew_geo = gpd.GeoDataFrame(breweries, crs = gdf.crs, geometry = breweries.geometry)

# Spatial join of brew_geo and gdf
brew_by_hood = gpd.sjoin(brew_geo, gdf, op = 'within')
print(brew_by_hood.head(2))

# Create permit_counts
brew_counts = brew_by_hood.groupby(['NAME']).size()
print(brew_counts)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
                                     id  ...      area
3         10-barrel-brewing-co-portland  ...  2.350873
20  back-pedal-brewing-company-portland  ...  2.350873

[2 rows x 31 columns]
NAME
ARBOR LODGE                                      1
ARGAY TERRACE                                    1
ASHCREEK                                         1
BOISE                                            2
BUCKMAN COMMUNITY ASSOCIATION                    8
CENTENNIAL COMMUNITY ASSOCIATION                 1
CONCORDIA                                        1
CRESTON-KENILWORTH                               1
CULLY ASSOCIATION OF NEIGHBORS                   1
ELIOT                                            3
HAZELWOOD                                        1
HILLSDALE                

Now we are getting somewhere! Time to combine this information with the area data from the neighborhoods (gdf) dataframe to get brewery density.

In [189]:
# Convert brew_counts to a DataFrame
breweries_df = brew_counts.to_frame()
print(breweries_df.head())

# Reset index and column names
breweries_df.reset_index(inplace=True)
breweries_df.columns = ['NAME', 'breweries']
print(breweries_df.head())

# Merge gdf and breweries_df: 
hoods_and_brews = pd.merge(gdf, breweries_df, how = 'outer', on = 'NAME')
print(hoods_and_brews.head(2))

                               0
NAME                            
ARBOR LODGE                    1
ARGAY TERRACE                  1
ASHCREEK                       1
BOISE                          2
BUCKMAN COMMUNITY ASSOCIATION  8
                            NAME  breweries
0                    ARBOR LODGE          1
1                  ARGAY TERRACE          1
2                       ASHCREEK          1
3                          BOISE          2
4  BUCKMAN COMMUNITY ASSOCIATION          8
   OBJECTID             NAME  ...      area breweries
0         1   CATHEDRAL PARK  ...  5.424298       NaN
1         2  UNIVERSITY PARK  ...  6.981457       NaN

[2 rows x 13 columns]


In [190]:
hoods_and_brews.fillna(value = 0, inplace = True)
print(hoods_and_brews.head(2))

   OBJECTID             NAME  ...      area breweries
0         1   CATHEDRAL PARK  ...  5.424298       0.0
1         2  UNIVERSITY PARK  ...  6.981457       0.0

[2 rows x 13 columns]


In [191]:
# Create brew_density column in hoods_and_brews
hoods_and_brews['brew_density'] = hoods_and_brews.apply(lambda row: row.breweries / row.area, axis = 1)

# Print the head of hoods_and_brews
print(hoods_and_brews.head())

   OBJECTID                            NAME  ... breweries brew_density
0         1                  CATHEDRAL PARK  ...       0.0     0.000000
1         2                 UNIVERSITY PARK  ...       0.0     0.000000
2         3                        PIEDMONT  ...       0.0     0.000000
3         4                        WOODLAWN  ...       1.0     0.258361
4         5  CULLY ASSOCIATION OF NEIGHBORS  ...       1.0     0.060311

[5 rows x 14 columns]


Now we have some normalized data with which we can create a choropleth map to visualize the actual density of breweries in Portland. 

In [192]:
# Center point for Portland is stored in the 'location' variable

# Create map
m = folium.Map(location=location, zoom_start=12)

# Build choropleth
m.choropleth(
    geo_data=hoods_and_brews,
    name='Neighborhoods',
    data=hoods_and_brews,
    columns=['NAME', 'brew_density'],
    key_on='feature.properties.NAME',
    fill_color='BuGn',
    fill_opacity=0.85,
    line_opacity=1.0,
    legend_name='Portland Neighborhoods - Breweries per square kilometer'
)

# Create center column for the centroid of each neighborhood
hoods_and_brews['center'] = hoods_and_brews.geometry.centroid

# Build markers at the center of each neighborhood with popups
for row in hoods_and_brews.iterrows():
    row_values = row[1] 
    center_point = row_values['center']
    loc = [center_point.y, center_point.x]
    popup = ('Neighborhood: ' + str(row_values['NAME']) + 
             ';  ' + 'Breweries per km^2: ' + str(row_values['brew_density']))
    marker = folium.Marker(location = loc, popup = popup)
    marker.add_to(m)

# Create LayerControl and add it to the map            
folium.LayerControl().add_to(m)

# Display the map
display(m)   

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



If we go through and check the group of brewery-dense neighborhoods, we can see that the Pearl District has the most breweries per square kilometer, at 2.13. However, there are a couple of close contenders in near proximity, and they all surround another neighborhood which has 0 breweries of its own. Buckman Community Association (South East of the Pearl District) has 1.31 Breweries per square kilometer, and Portland Downtown (South of the Pearl District) has 1.01 breweries per square kilometer. 

As mentioned above, there is a fourth neighborhood which is surrounded by the three mentioned above which has zero breweries of its own. The Old Town Community Association has zero breweries, but would provide easy walking to three of the most brewery-dense neighborhoods in all of Portland.

In [193]:
print(hoods_and_brews.head())

   OBJECTID  ...                       center
0         1  ...  POINT (-122.75732 45.58737)
1         2  ...  POINT (-122.73008 45.57635)
2         3  ...  POINT (-122.67042 45.57644)
3         4  ...  POINT (-122.65304 45.57257)
4         5  ...  POINT (-122.60151 45.56375)

[5 rows x 15 columns]
